# Лабораторная работа 8. Выбор оптимального классификатора

In [19]:
%matplotlib inline

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report

import warnings
warnings.filterwarnings('ignore')

В этой лабораторной работе вам потребуется выбрать наилучший классификатор с оптимальными параметрами для задачи про пассажиров ["Титаника"](https://ru.wikipedia.org/wiki/Титаник).

__Задание 1.__  
Загрузите данные (см. предыдущую лабораторную работу).

In [20]:
train_data = pd.read_csv('input/train.csv')
test_data = pd.read_csv('input/test.csv')

__Задание 2.__  
Проведите предобработку данных (см. предыдущую лабораторную работу).

Данные были предобработаны в лабораторной работе № 7

__Задание 3.__  
Примените масштабирование признаков (`StandardScaler`, `MinMaxScaler`).

In [21]:
numeric_features = ['Age', 'SibSp', 'Parch', 'Fare']

# Создание экземпляра StandardScaler
scaler_standard = StandardScaler()

# Масштабирование числовых признаков в обучающих данных
train_data[numeric_features] = scaler_standard.fit_transform(train_data[numeric_features])

# Масштабирование числовых признаков в тестовых данных
test_data[numeric_features] = scaler_standard.transform(test_data[numeric_features])

print("Обучающие данные после StandardScaler:")
print(train_data[numeric_features].head())
print("\nСтатистика после StandardScaler:")
print(train_data[numeric_features].describe())

# Создание экземпляра MinMaxScaler
scaler_minmax = MinMaxScaler()

# Масштабирование числовых признаков в обучающих данных
train_data[numeric_features] = scaler_minmax.fit_transform(train_data[numeric_features])

# Масштабирование числовых признаков в тестовых данных
test_data[numeric_features] = scaler_minmax.transform(test_data[numeric_features])

print("\nОбучающие данные после MinMaxScaler:")
print(train_data[numeric_features].head())
print("\nСтатистика после MinMaxScaler:")
print(train_data[numeric_features].describe())

Обучающие данные после StandardScaler:
        Age     SibSp     Parch      Fare
0 -0.095928  0.432793 -0.473674 -0.502445
1  0.806944  0.432793 -0.473674  0.786845
2  0.129790 -0.474545 -0.473674 -0.488854
3  0.637655  0.432793 -0.473674  0.420730
4  0.637655 -0.474545 -0.473674 -0.486337

Статистика после StandardScaler:
                Age         SibSp         Parch          Fare
count  8.910000e+02  8.910000e+02  8.910000e+02  8.910000e+02
mean   3.588600e-17  4.386066e-17  5.382900e-17  3.987333e-18
std    1.000562e+00  1.000562e+00  1.000562e+00  1.000562e+00
min   -1.365592e+00 -4.745452e-01 -4.736736e-01 -6.484217e-01
25%   -9.988002e-01 -4.745452e-01 -4.736736e-01 -4.891482e-01
50%    1.693075e-02 -4.745452e-01 -4.736736e-01 -3.573909e-01
75%    6.376552e-01  4.327934e-01 -4.736736e-01 -2.424635e-02
max    3.176983e+00  6.784163e+00  6.974147e+00  9.667167e+00

Обучающие данные после MinMaxScaler:
        Age  SibSp  Parch      Fare
0  0.279503  0.125    0.0  0.014151
1  0.47

__Задание 4.__  
Примените различные преобразования признаков (`PolynomialFeatures`).

In [22]:
# Создание экземпляра PolynomialFeatures
poly = PolynomialFeatures(degree=2, include_bias=False)

# Заполнение пропущенных значений средними значениями
numeric_features = ['Age', 'SibSp', 'Parch', 'Fare']
train_data[numeric_features] = train_data[numeric_features].fillna(train_data[numeric_features].mean())
test_data[numeric_features] = test_data[numeric_features].fillna(test_data[numeric_features].mean())

# Преобразование числовых признаков в обучающих данных
X_train_poly = poly.fit_transform(train_data[numeric_features])

# Преобразование числовых признаков в тестовых данных
X_test_poly = poly.transform(test_data[numeric_features])

# Получение названий новых признаков
new_feature_names = poly.get_feature_names_out(numeric_features)

# Создание DataFrame с новыми признаками
X_train_poly_df = pd.DataFrame(X_train_poly, columns=new_feature_names)
X_test_poly_df = pd.DataFrame(X_test_poly, columns=new_feature_names)

# Объединение новых признаков с исходными данными
train_data = pd.concat([train_data.drop(numeric_features, axis=1), X_train_poly_df], axis=1)
test_data = pd.concat([test_data.drop(numeric_features, axis=1), X_test_poly_df], axis=1)

print("Обучающие данные после преобразования:")
print(train_data.head())
print("\nФорма обучающих данных:", train_data.shape)
print("\nНовые признаки:")
print(new_feature_names)

Обучающие данные после преобразования:
   Unnamed: 0  PassengerId  Pclass  Pclass_1  Pclass_2  Pclass_3  Sex_female  \
0           0            1       3         0         0         1           0   
1           1            2       1         1         0         0           1   
2           2            3       3         0         0         1           1   
3           3            4       1         1         0         0           1   
4           4            5       3         0         0         1           0   

   Sex_male  Age_categories_Missing  Age_categories_Infant  ...     Age^2  \
0         1                       0                      0  ...  0.078122   
1         0                       0                      0  ...  0.228733   
2         0                       0                      0  ...  0.108368   
3         0                       0                      0  ...  0.194476   
4         1                       0                      0  ...  0.194476   

   Age SibSp  Age

__Задание 5.__  
Обучите несколько классификаторов, в том числе:  
1. Логистическую регрессию (`LogisticRegression`).
1. Метод опорных векторов (`SVC`).
1. Метод *k* ближайших соседей (`KNeighborsClassifier`).
1. Наивный байесовский классификатор (`MultinomialNB`).
1. Деревья решений (`DecisionTreeClassifier`).
1. Случайный лес (`RandomForestClassifier`).
1. AdaBoost (`AdaBoost`).
1. Градиентный бустинг (`GradientBoostingClassifier`).

Для обучения и проверки качества можно использовать функцию `train_test_split()`.

In [23]:
# Подготовка данных для обучения
X = train_data.drop('Survival', axis=1)
y = train_data['Survival']

# Создание списка классификаторов
classifiers = {
    'Логистическая регрессия': LogisticRegression(random_state=42),
    'Метод опорных векторов': SVC(random_state=42),
    'K-ближайших соседей': KNeighborsClassifier(),
    'Наивный Байес': MultinomialNB(),
    'Дерево решений': DecisionTreeClassifier(random_state=42),
    'Случайный лес': RandomForestClassifier(random_state=42),
    'AdaBoost': AdaBoostClassifier(random_state=42),
    'Градиентный бустинг': GradientBoostingClassifier(random_state=42)
}

# Обучение и оценка каждого классификатора
results = {}
for name, clf in classifiers.items():
    print(f"\n{'='*50}")
    print(f"Обучение {name}...")
    
    # Обучение модели
    clf.fit(X, y)
    
    # Предсказание на обучающих данных
    y_train_pred = clf.predict(X)
    train_accuracy = accuracy_score(y, y_train_pred)
    
    print(f"\nМетрики качества на обучающих данных:")
    print(f"Точность (Accuracy): {train_accuracy:.4f}")
    print("\nОтчет о классификации:")
    print(classification_report(y, y_train_pred))
    
    # Вывод вероятностей для каждого класса
    if hasattr(clf, 'predict_proba'):
        y_proba = clf.predict_proba(X)
        print("\nСредние вероятности классов:")
        print(f"Класс 0 (не выжил): {y_proba[:, 0].mean():.4f}")
        print(f"Класс 1 (выжил): {y_proba[:, 1].mean():.4f}")
    
    # Для деревьев и лесов выведем важность признаков
    if hasattr(clf, 'feature_importances_'):
        feature_importance = pd.DataFrame({
            'Признак': X.columns,
            'Важность': clf.feature_importances_
        }).sort_values('Важность', ascending=False)
        print("\nТоп-5 важных признаков:")
        print(feature_importance.head())
    
    # Сохранение результатов для сравнения
    results[name] = train_accuracy

# Вывод сводной таблицы результатов
print("\nСводная таблица результатов:")
results_df = pd.DataFrame.from_dict(results, orient='index', columns=['Точность'])
print(results_df.sort_values('Точность', ascending=False))


Обучение Логистическая регрессия...

Метрики качества на обучающих данных:
Точность (Accuracy): 1.0000

Отчет о классификации:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       577
           1       1.00      1.00      1.00       314

    accuracy                           1.00       891
   macro avg       1.00      1.00      1.00       891
weighted avg       1.00      1.00      1.00       891


Средние вероятности классов:
Класс 0 (не выжил): 0.6476
Класс 1 (выжил): 0.3524

Обучение Метод опорных векторов...

Метрики качества на обучающих данных:
Точность (Accuracy): 0.6476

Отчет о классификации:
              precision    recall  f1-score   support

           0       0.65      1.00      0.79       577
           1       0.00      0.00      0.00       314

    accuracy                           0.65       891
   macro avg       0.32      0.50      0.39       891
weighted avg       0.42      0.65      0.51       891


Обучение 

__Задание 6.__  
При помощи `Pipeline` и `GridSearchCV` выберите оптимальную архитектуру:
1. Метод масштабирования.
1. Степень полинома в `PolynomialFeatures`.
1. Параметры классификаторов (в том числе, параметры регуляризации).

Заносите в таблицу Excel результаты тестирования (варианты параметров, оценки качества).

In [24]:
# Ваш код здесь

__Задание 7.__  
1. Выберите несколько лучших классификаторов (от 3 до 10).
1. Обучите выбранные классификаторы на всех доступных размеченных данных.
1. Получите результаты предсказания для тестовых данных.
1. Отправьте результаты на сервер [Kaggle](https://ru.wikipedia.org/wiki/Титаник).

In [25]:
# Ваш код здесь